# 0.0 Data preparation

# Preparation

Imports libraries

Loads data

In [ ]:
%matplotlib inline

print("0.0 data preparation")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import re
import json
import datetime
import random

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain
from scipy.stats import ttest_ind

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
processRaw20190603 = not ('raw20190603' in globals())

### common variables

In [ ]:
dataFilesNamesStem = "2019-06-03_event_raw"

dataFolderPath = "../Data/"
processedSuffix = "-processed"
csvEncoding = 'utf-8'
csvSuffix = '.csv'

In [ ]:
questionCount = 12
questionArrayInt = [i+1 for i in range(questionCount)]
questionArrayStr = sorted(["Q" + "{0:0=2d}".format(i) for i in questionArrayInt])
questionArrayStr

In [ ]:
raw20190603Path = dataFolderPath + dataFilesNamesStem + csvSuffix

# Functions

## Date parser
Need to fix dates, incorrect time zone

In [ ]:
str(123)

## Loading

In [ ]:
raw20190603Path

In [ ]:
dateparseGForm = lambda x: pd.Timestamp(str(x).split(' GMT')[0], tz='Asia/Kolkata') #.tz_convert('utc')
raw20190603  = pd.read_csv(\
                            raw20190603Path,\
                            dtype=str,\
                            parse_dates=['Timestamp'],\
                            date_parser=dateparseGForm\
                         )

In [ ]:
raw20190603

In [ ]:
raw20190603.columns

In [ ]:
timestampQuestion = raw20190603.columns[0]
nameQuestion = raw20190603.columns[1]
emailQuestion = raw20190603.columns[2]
gameQuestion = raw20190603.columns[3]

In [ ]:
shortQuestions = pd.Series(index=questionArrayStr, data=raw20190603.columns[4:])
shortQuestions

In [ ]:
indexedQuestions = pd.Series(index=range(1,13), data=raw20190603.columns[4:])
indexedQuestions

In [ ]:
games = raw20190603[gameQuestion].unique()
games

In [ ]:
gameDrBugTitle = 'Dr Bug: Microbe Mayhem'
gameSuperbugsTitle = 'Superbugs: the Game'
gameFungalTitle = 'Fungal Invasion'